# Set up


In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd Path/to/your/directory

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install timm
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install wand
!pip install torchmetrics -q
!pip install tqdm -q

# TreeVision


## Train

In [ ]:
import os

dataset_name = input("Choose a dataset: ").lower()  # london, yosemite_512, yosemite_1536
DATASET_PATH = os.path.join("datasets", dataset_name)

Choose a dataset: london


In [ ]:
import wandb

config = dict(
    epochs=1,
    batch_size=1,
    lr=1e-4,
    decay=5*1e-4,
    workers=1,
    pre=None,
    id=wandb.util.generate_id(),
    dataset_path=DATASET_PATH,
    dataset_name=dataset_name)

print(config["id"])

In [ ]:
from train import pipeline

model = pipeline(config)

## Validate

In [ ]:
import os

import torch
import torch.nn as nn
from model import TreeVision
from val import validate
from utils import prepare_datasets


checkpoint_path = "weights/Path/to/your/checkpoint.pth"
dataset_name = input("Choose a dataset: ").lower()  # london, yosemite_512, yosemite_1536
dataset_path = os.path.join("datasets", dataset_name)

model = TreeVision()
model = model.cuda()
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['state_dict'])

_, val_list = prepare_datasets(dataset_path, dataset_name)

criterion = nn.MSELoss(size_average=False).cuda()

config = dict(
    batch_size=1,
    workers=1,
    dataset_path=dataset_path,
    dataset_name=dataset_name)

validate(val_list, model, criterion, config)


## Inference (Yosemite)

In [ ]:
from matplotlib import cm as c
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import torch
from scipy.ndimage import gaussian_filter
from utils import gaussian_filter_density
from model import TreeVision

img_path = 'datasets/yosemite_512/zone_A/images/IMG_0.jpg'
weight_path = 'weights/yosemite_512_best.pth'

labels = []
with open(img_path.replace("images", "labels").replace("jpg", "txt"), "r") as f:
    for tree in f.readlines():
        x, y = tree.split(" ")
        labels.append((float(x), float(y)))

print("Original Image")
print("Original Count : ", len(labels))
img = plt.imread(img_path)
plt.imshow(img)
plt.show()

model = TreeVision()
model = model.cuda()
checkpoint = torch.load(weight_path)
model.load_state_dict(checkpoint['state_dict'])

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                           std=[0.229, 0.224, 0.225]),])
img = transform(Image.open(img_path).convert('RGB')).cuda()

output = model(img.unsqueeze(0))
print("Predicted Count : ",int(output.detach().cpu().sum().numpy()))
temp = np.asarray(output.detach().cpu().reshape(output.detach().cpu().shape[2],output.detach().cpu().shape[3]))
plt.imshow(temp,cmap = c.jet)
plt.show()

## Inference (London)

In [ ]:
from matplotlib import cm as c
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
from model import TreeVision
import scipy.io as sio
import os
import torch

img_path = 'datasets/london/test/images/IMG_167.jpg'
weight_path = "weights/london_best.pth"

print("Original Image")
img = plt.imread(img_path)
plt.imshow(img)
plt.show()

name = os.path.basename(img_path).split('.')[0]
gd_path = os.path.join(os.path.dirname(img_path).replace("images", "labels"), 'GT_{}.mat'.format(name))
keypoints = sio.loadmat(gd_path)['image_info'][0][0][0][0][0]
gauss_path = os.path.join(os.path.dirname(img_path).replace("images", "labels"), '{}_densitymap.npy'.format(name))
gauss_im = torch.from_numpy(np.load(gauss_path)).float()
plt.imshow(gauss_im, cmap = c.jet)

print("Original Count : ", len(keypoints))
plt.show()

model = TreeVision()
model = model.cuda()
checkpoint = torch.load(weight_path)
model.load_state_dict(checkpoint['state_dict'])

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                           std=[0.229, 0.224, 0.225]),])
img = transform(Image.open(img_path).convert('RGB')).cuda()

output = model(img.unsqueeze(0))
print("Predicted Count : ",int(output.detach().cpu().sum().numpy()))
temp = np.asarray(output.detach().cpu().reshape(output.detach().cpu().shape[2],output.detach().cpu().shape[3]))
plt.imshow(temp,cmap = c.jet)
plt.show()